In [ ]:
"""
把从上交所和深交所下载的年报pdf，提取文本

为了加速，采用多进程并发方式
"""

In [20]:
#!jupyter nbconvert --to script pdf_to_txt.ipynb
import os
def pdf_to_txt(pdf_file,txt_dir='./'):
    import subprocess
    import os
    try:
        txt_filename=txt_dir+os.path.basename(pdf_file).replace('pdf','txt')
        fsize=0
        try:
            if os.path.exists(txt_filename):
                fsize = os.path.getsize(txt_filename)
                print(f'【进程{os.getpid()}】file {txt_filename} size is {int(fsize/1024)}kb')
        except Exception as e:
            print(f'【进程{os.getpid()}】getsize exception {txt_filename} ',e)
         
        if fsize > 1024*10:
            print(f'【进程{os.getpid()}】jump...')

        fsize = 0
        try:
            if os.path.exists(pdf_file):
                fsize = os.path.getsize(pdf_file)
        except Exception as e:
            return 
#         out_bytes = subprocess.check_output(['which','pdf2txt.py'])
#         pdf2txt=out_bytes.decode("utf-8").replace('\n','')
#         if not pdf2txt:
#             print('pdf2txt.py 不存在，程序退出<c')
#             quit()
#         subprocess.run(["python", pdf2txt, "-o",txt_filename ,pdf_file])
        t0 = time.time()
        print(f'【进程{os.getpid()}】开始转换文件：{pdf_file}')
        subprocess.run(["pdf2txt.py", "-o",txt_filename ,pdf_file])
        print(f'【进程{os.getpid()}】转换文件成功，用时={int(time.time()-t0)}秒, 转换速度={int(fsize/1024/(time.time()-t0))}KB/秒')
    except subprocess.CalledProcessError as e:
        out_bytes = e.output       # Output generated before error
        code      = e.returncode   # Return code
        print(out_bytes,code)
    except:
        out_bytes = e.output       # Output generated before error
        code      = e.returncode   # Return code
        print("***=",out_bytes,code)
     

    


# pdf_file=pdf_dir+'000780_2006.pdf' 
# pdf_to_txt(pdf_file)

In [21]:
# find files

def find_files(pdf_dir):
    return [pdf_dir+f for f in os.listdir(pdf_dir) if f.endswith('.pdf')]

pdf_dir ='../scraping/pdf/'
pdf_files = find_files(pdf_dir)
print(len(pdf_files))
print(pdf_files[:4])

2046
['../scraping/pdf/000503_2011.pdf', '../scraping/pdf/000429_2002.pdf', '../scraping/pdf/000008_2017.pdf', '../scraping/pdf/000155_2011.pdf']


In [22]:


base_dir='../scraping/content/'
import time  
from multiprocessing import Process, Queue
from tqdm import tqdm

q = Queue()

def f(q: Queue):
    while True:
        pdf_file = q.get()
        pdf_to_txt(pdf_file,base_dir)



            
def loop_task():
    import time  
    for pdf_file in tqdm(pdf_files):
        if q.qsize()> 10:
            print("sleep for queue is not empty")
            time.sleep(60)  # sleep 10秒
        else:
            q.put(pdf_file)


# 启动四个进程进行处理
for i in range(2):
    p = Process(target=f, args=(q,))
    p.start()

loop_task()

    







  0%|          | 0/2046 [00:00<?, ?it/s]

【进程26443】file ../scraping/content/000503_2011.txt size is 175kb
【进程26444】file ../scraping/content/000429_2002.txt size is 190kb
【进程26443】jump...
【进程26444】jump...
【进程26443】开始转换文件：../scraping/pdf/000503_2011.pdf
【进程26444】开始转换文件：../scraping/pdf/000429_2002.pdf
sleep for queue is not empty
【进程26444】转换文件成功，用时(秒):7, 速度=33KB/秒
【进程26444】file ../scraping/content/000008_2017.txt size is 113kb
【进程26444】jump...
【进程26444】开始转换文件：../scraping/pdf/000008_2017.pdf








  1%|          | 12/2046 [01:00<2:49:39,  5.00s/it]

sleep for queue is not empty


KeyboardInterrupt: 

Process Process-15:
Traceback (most recent call last):
  File "<ipython-input-20-c206e0925569>", line 33, in pdf_to_txt
    subprocess.run(["pdf2txt.py", "-o",txt_filename ,pdf_file])
  File "/root/miniconda3/lib/python3.7/subprocess.py", line 474, in run
    stdout, stderr = process.communicate(input, timeout=timeout)
  File "/root/miniconda3/lib/python3.7/subprocess.py", line 931, in communicate
    self.wait()
  File "/root/miniconda3/lib/python3.7/subprocess.py", line 990, in wait
    return self._wait(timeout=timeout)
  File "/root/miniconda3/lib/python3.7/subprocess.py", line 1624, in _wait
    (pid, sts) = self._try_wait(0)
  File "/root/miniconda3/lib/python3.7/subprocess.py", line 1582, in _try_wait
    (pid, sts) = os.waitpid(self.pid, wait_flags)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Process Process-16:
Traceback (most recent call last):
Traceback (most recent call last):
  File "<ipython-input-20-c206e0925569>", line 33, in 